<div style="text-align: center; font-size: 30pt; font-weight: bold; margin: 1em 0em 1em 0em">Auto-Encoders v. Variational Auto-Encoders</div>

In [1]:
import sys
import os

In [2]:
sys.path.append(os.path.abspath('../autoencoders'))

In [3]:
# "Magic" commands for automatic reloading of module, perfect for prototyping
%reload_ext autoreload
%autoreload 2

import autoencoders

# The dataset

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets

In [5]:
root = '../data'
    
trans = transforms.Compose([
    # transforms.ToTensor(), 
    # transforms.ToPILImage(),
    # transforms.Resize((8, 8)),
    transforms.ToTensor()
    # transforms.Normalize((0.5,), (1.0,))
])

# if not exist, download mnist dataset
train_set = datasets.MNIST(root=root, train=True, transform=trans, download=True)
test_set = datasets.MNIST(root=root, train=False, transform=trans, download=True)

batch_size = 100

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

print('>> total trainning batch number: {}'.format(len(train_loader)))
print('>> total testing batch number: {}'.format(len(test_loader)))

>> total trainning batch number: 600
>> total testing batch number: 100


# Auto-encoder

In [4]:
model = autoencoders.AutoEncoder()
print(model)

AutoEncoder(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=12, bias=True)
  (fc4): Linear(in_features=12, out_features=2, bias=True)
  (fc5): Linear(in_features=2, out_features=12, bias=True)
  (fc6): Linear(in_features=12, out_features=50, bias=True)
  (fc7): Linear(in_features=50, out_features=128, bias=True)
  (fc8): Linear(in_features=128, out_features=784, bias=True)
)


## Loss and training routine

In [7]:
device = torch.device("cpu")

In [8]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [9]:
log_interval = 200

In [10]:
import torch.nn.functional as F

In [11]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x):
    
    # reconstruction = F.binary_cross_entropy(recon_x, x.view(-1, 28*28), reduction='sum')
    reconstruction = F.mse_loss(recon_x, x.view(-1, 28*28), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    # KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return reconstruction


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch = model(data)
        loss = loss_function(recon_batch, data)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('>> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch = model(data)
            test_loss += loss_function(recon_batch, data).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(100, 1, 28, 28)[:n]])
                utils.save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('>> Test set loss: {:.4f}'.format(test_loss))

## Training

In [15]:
with torch.no_grad():
    for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch = model(data)
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(100, 1, 28, 28)[:n]])
                utils.save_image(comparison.cpu(),
                         'results/reconstruction_0.png', nrow=n)
                
    sample = 3 * torch.randn(64, 2).to(device)
    sample = model.decode(sample).cpu()
    utils.save_image(sample.view(64, 1, 28, 28),
               'results/sample_0.png')

In [19]:
import numpy as np

In [72]:
space = np.array([[x, y] for y in np.linspace(0, 5, 16) for x in np.linspace(0, 5, 16)], dtype=np.float)
space = torch.from_numpy(space).type(torch.FloatTensor)

In [73]:
epoch += 1
with torch.no_grad():
    # sample = torch.randn(64, 2).to(device)
    sample = space
    sample = model.decode(sample).cpu()
    utils.save_image(sample.view(16 * 16, 1, 28, 28),
               'results/sample_' + str(epoch) + '.png', nrow=16)

In [74]:
for epoch in range(100, 200):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        # sample = torch.randn(64, 2).to(device)
        sample = space + .1 * torch.randn(16 * 16, 2).to(device)
        sample = model.decode(sample).cpu()
        utils.save_image(sample.view(16 * 16, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png', nrow=16)

Train Epoch: 100 [0/60000 (0%)]	Loss: 27.434194
Train Epoch: 100 [20000/60000 (33%)]	Loss: 27.653030
Train Epoch: 100 [40000/60000 (67%)]	Loss: 28.132900
>> Epoch: 100 Average loss: 28.0925
>> Test set loss: 29.1073
Train Epoch: 101 [0/60000 (0%)]	Loss: 26.247185
Train Epoch: 101 [20000/60000 (33%)]	Loss: 26.054910
Train Epoch: 101 [40000/60000 (67%)]	Loss: 27.263174
>> Epoch: 101 Average loss: 27.8610
>> Test set loss: 28.4108
Train Epoch: 102 [0/60000 (0%)]	Loss: 26.615098
Train Epoch: 102 [20000/60000 (33%)]	Loss: 25.315955
Train Epoch: 102 [40000/60000 (67%)]	Loss: 27.422822
>> Epoch: 102 Average loss: 27.5723
>> Test set loss: 27.8513
Train Epoch: 103 [0/60000 (0%)]	Loss: 29.024128
Train Epoch: 103 [20000/60000 (33%)]	Loss: 27.392444
Train Epoch: 103 [40000/60000 (67%)]	Loss: 27.526680
>> Epoch: 103 Average loss: 27.5940
>> Test set loss: 27.8839
Train Epoch: 104 [0/60000 (0%)]	Loss: 24.922517
Train Epoch: 104 [20000/60000 (33%)]	Loss: 26.165518
Train Epoch: 104 [40000/60000 (67%)

Train Epoch: 138 [20000/60000 (33%)]	Loss: 28.478911
Train Epoch: 138 [40000/60000 (67%)]	Loss: 28.039736
>> Epoch: 138 Average loss: 27.2994
>> Test set loss: 27.8493
Train Epoch: 139 [0/60000 (0%)]	Loss: 26.656230
Train Epoch: 139 [20000/60000 (33%)]	Loss: 26.371052
Train Epoch: 139 [40000/60000 (67%)]	Loss: 28.662974
>> Epoch: 139 Average loss: 27.3739
>> Test set loss: 27.8157
Train Epoch: 140 [0/60000 (0%)]	Loss: 25.846450
Train Epoch: 140 [20000/60000 (33%)]	Loss: 28.036323
Train Epoch: 140 [40000/60000 (67%)]	Loss: 27.240313
>> Epoch: 140 Average loss: 27.2803
>> Test set loss: 27.7956
Train Epoch: 141 [0/60000 (0%)]	Loss: 29.586353
Train Epoch: 141 [20000/60000 (33%)]	Loss: 26.635659
Train Epoch: 141 [40000/60000 (67%)]	Loss: 26.787249
>> Epoch: 141 Average loss: 27.7083
>> Test set loss: 27.9262
Train Epoch: 142 [0/60000 (0%)]	Loss: 26.102864
Train Epoch: 142 [20000/60000 (33%)]	Loss: 25.811187
Train Epoch: 142 [40000/60000 (67%)]	Loss: 26.763083
>> Epoch: 142 Average loss: 27

Train Epoch: 176 [20000/60000 (33%)]	Loss: 25.644033
Train Epoch: 176 [40000/60000 (67%)]	Loss: 26.782109
>> Epoch: 176 Average loss: 26.9754
>> Test set loss: 27.5705
Train Epoch: 177 [0/60000 (0%)]	Loss: 26.766528
Train Epoch: 177 [20000/60000 (33%)]	Loss: 29.254583
Train Epoch: 177 [40000/60000 (67%)]	Loss: 23.948616
>> Epoch: 177 Average loss: 26.9804
>> Test set loss: 27.3789
Train Epoch: 178 [0/60000 (0%)]	Loss: 25.275842
Train Epoch: 178 [20000/60000 (33%)]	Loss: 25.949575
Train Epoch: 178 [40000/60000 (67%)]	Loss: 27.896028
>> Epoch: 178 Average loss: 26.8892
>> Test set loss: 27.7058
Train Epoch: 179 [0/60000 (0%)]	Loss: 26.909988
Train Epoch: 179 [20000/60000 (33%)]	Loss: 26.633896
Train Epoch: 179 [40000/60000 (67%)]	Loss: 25.283594
>> Epoch: 179 Average loss: 26.9147
>> Test set loss: 27.4598
Train Epoch: 180 [0/60000 (0%)]	Loss: 26.766208
Train Epoch: 180 [20000/60000 (33%)]	Loss: 28.998052
Train Epoch: 180 [40000/60000 (67%)]	Loss: 28.345520
>> Epoch: 180 Average loss: 27

# Variational Auto-Encoder

In [23]:
vae = autoencoders.VariationalAutoEncoder()
print(vae)

VariationalAutoEncoder(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=12, bias=True)
  (fc41): Linear(in_features=12, out_features=2, bias=True)
  (fc42): Linear(in_features=12, out_features=2, bias=True)
  (fc5): Linear(in_features=2, out_features=12, bias=True)
  (fc6): Linear(in_features=12, out_features=50, bias=True)
  (fc7): Linear(in_features=50, out_features=128, bias=True)
  (fc8): Linear(in_features=128, out_features=784, bias=True)
)


## Loss and training routine

In [24]:
device = torch.device("cpu")

In [25]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate, weight_decay=1e-5)

In [26]:
log_interval = 200

In [27]:
import torch.nn.functional as F

In [28]:
from tensorboardX import SummaryWriter
writer = SummaryWriter()

In [29]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function_vae(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, log_variance = vae(data)
        loss = loss_function_vae(recon_batch, data, mu, log_variance)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        # if batch_idx % log_interval == 0:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(data), len(train_loader.dataset),
        #         100. * batch_idx / len(train_loader),
        #         loss.item() / len(data)))

    print('>> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))
    
    writer.add_scalar('data/train-loss', train_loss / len(train_loader.dataset), epoch)


def test(epoch):
    
    vae.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, log_variance = vae(data)
            test_loss += loss_function_vae(recon_batch, data, mu, log_variance).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(100, 1, 28, 28)[:n]])
                # utils.save_image(comparison.cpu(),
                #          'results/vae-reconstruction_' + str(epoch) + '.png', nrow=n)
                
                writer.add_image('reconstruction', comparison.cpu(), epoch)

    test_loss /= len(test_loader.dataset)
    print('>> Test set loss: {:.4f}'.format(test_loss))
    
    writer.add_scalar('data/test-loss', test_loss, epoch)

## Training

In [30]:
import numpy as np

In [31]:
space = np.array([[x, y] for y in np.linspace(-1.5, 1.5, 16) for x in np.linspace(-1.5, 1.5, 16)], dtype=np.float)
space = torch.from_numpy(space).type(torch.FloatTensor)

In [32]:
for epoch in range(1, 100):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        # sample = torch.randn(64, 2).to(device)
        sample = space # + .05 * torch.randn(16 * 16, 2).to(device)
        sample = vae.decode(sample).cpu()
        utils.save_image(sample.view(16 * 16, 1, 28, 28),
                   'results/vae-sample_' + str(epoch) + '.png', nrow=16)
        writer.add_image('sample', utils.make_grid(sample.view(16 * 16, 1, 28, 28), nrow=16), epoch)

>> Epoch: 1 Average loss: 207.4965
>> Test set loss: 183.7865
>> Epoch: 2 Average loss: 173.9688
>> Test set loss: 168.5135
>> Epoch: 3 Average loss: 165.8932
>> Test set loss: 163.6707
>> Epoch: 4 Average loss: 161.1973
>> Test set loss: 159.0849
>> Epoch: 5 Average loss: 157.4966
>> Test set loss: 156.1624
>> Epoch: 6 Average loss: 155.0789
>> Test set loss: 154.2824
>> Epoch: 7 Average loss: 153.4772
>> Test set loss: 153.5132
>> Epoch: 8 Average loss: 152.2615
>> Test set loss: 151.9901
>> Epoch: 9 Average loss: 151.1153
>> Test set loss: 151.1691
>> Epoch: 10 Average loss: 150.1952
>> Test set loss: 150.4310
>> Epoch: 11 Average loss: 149.4633
>> Test set loss: 149.4782
>> Epoch: 12 Average loss: 148.7509
>> Test set loss: 148.9511
>> Epoch: 13 Average loss: 148.2019
>> Test set loss: 148.4081
>> Epoch: 14 Average loss: 147.6030
>> Test set loss: 147.8209
>> Epoch: 15 Average loss: 147.0881
>> Test set loss: 147.1794
>> Epoch: 16 Average loss: 146.7729
>> Test set loss: 147.1360
>

KeyboardInterrupt: 